In [1]:
import pandas as pd
import numpy as np
import random
from multiprocessing import Process, Manager
from itertools import repeat
from tqdm import trange

In [2]:
# Load data
data = pd.read_csv('../input/mlbplaybyplay2010s/all2018.csv', low_memory=False)

In [3]:
# Leave only the PA data
pa_events = [2, 3, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
data = data[data['EVENT_CD'].isin(pa_events)].reset_index(drop=True)

In [4]:
pd.options.display.max_columns = 120
data

,GAME_ID,AWAY_TEAM_ID,INN_CT,BAT_HOME_ID,OUTS_CT,BALLS_CT,STRIKES_CT,PITCH_SEQ_TX,AWAY_SCORE_CT,HOME_SCORE_CT,BAT_ID,BAT_HAND_CD,RESP_BAT_ID,RESP_BAT_HAND_CD,PIT_ID,PIT_HAND_CD,RESP_PIT_ID,RESP_PIT_HAND_CD,POS2_FLD_ID,POS3_FLD_ID,POS4_FLD_ID,POS5_FLD_ID,POS6_FLD_ID,POS7_FLD_ID,POS8_FLD_ID,POS9_FLD_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,EVENT_TX,LEADOFF_FL,PH_FL,BAT_FLD_CD,BAT_LINEUP_ID,EVENT_CD,BAT_EVENT_FL,AB_FL,H_FL,SH_FL,SF_FL,EVENT_OUTS_CT,DP_FL,TP_FL,RBI_CT,WP_FL,PB_FL,FLD_CD,BATTEDBALL_CD,BUNT_FL,FOUL_FL,BATTEDBALL_LOC_TX,ERR_CT,ERR1_FLD_CD,ERR1_CD,ERR2_FLD_CD,ERR2_CD,ERR3_FLD_CD,ERR3_CD,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BAT_PLAY_TX,RUN1_PLAY_TX,RUN2_PLAY_TX,RUN3_PLAY_TX,RUN1_SB_FL,RUN2_SB_FL,RUN3_SB_FL,RUN1_CS_FL,RUN2_CS_FL,RUN3_CS_FL,RUN1_PK_FL,RUN2_PK_FL,RUN3_PK_FL,RUN1_RESP_PIT_ID,RUN2_RESP_PIT_ID,RUN3_RESP_PIT_ID,GAME_NEW_FL,GAME_END_FL,PR_RUN1_FL,PR_RUN2_FL,PR_RUN3_FL,REMOVED_FOR_PR_RUN1_ID,REMOVED_FOR_PR_RUN2_ID,REMOVED_FOR_PR_RUN3_ID,REMOVED_FOR_PH_BAT_ID,REMOVED_FOR_PH_BAT_FLD_CD,PO1_FLD_CD,PO2_FLD_CD,PO3_FLD_CD,ASS1_FLD_CD,ASS2_FLD_CD,ASS3_FLD_CD,ASS4_FLD_CD,ASS5_FLD_CD,EVENT_ID
0,ANA201804020,CLE,1,0,0,2,2,BCBSX,0,0,lindf001,L,lindf001,L,ramij002,R,ramij002,R,maldm001,martj007,cozaz001,valbl001,simma001,uptoj001,troum001,calhk001,NaN,NaN,NaN,8/L+,T,F,6,1,2,T,T,0,F,F,1,F,F,0,F,F,8,L,F,F,NaN,0,0,N,0,N,0,N,0,0,0,0,8.0,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,T,F,F,F,F,NaN,NaN,NaN,NaN,0,8,0,0,0,0,0,0,0,1
1,ANA201804020,CLE,1,0,1,1,2,CBCX,0,0,kipnj001,L,kipnj001,L,ramij002,R,ramij002,R,maldm001,martj007,cozaz001,valbl001,simma001,uptoj001,troum001,calhk001,NaN,NaN,NaN,31/G,F,F,4,2,2,T,T,0,F,F,1,F,F,0,F,F,3,G,F,F,NaN,0,0,N,0,N,0,N,0,0,0,0,31.0,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,1,0,0,3,0,0,0,0,2
2,ANA201804020,CLE,1,0,2,1,2,CBSFFX,0,0,ramij003,L,ramij003,L,ramij002,R,ramij002,R,maldm001,martj007,cozaz001,valbl001,simma001,uptoj001,troum001,calhk001,NaN,NaN,NaN,31/G,F,F,5,3,2,T,T,0,F,F,1,F,F,0,F,F,3,G,F,F,NaN,0,0,N,0,N,0,N,0,0,0,0,31.0,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,1,0,0,3,0,0,0,0,3
3,ANA201804020,CLE,1,1,0,0,0,X,0,0,cozaz001,R,cozaz001,R,clevm001,R,clevm001,R,gomey001,alony001,kipnj001,ramij003,lindf001,naqut001,zimmb001,chisl001,NaN,NaN,NaN,8/F,T,F,4,1,2,T,T,0,F,F,1,F,F,0,F,F,8,F,F,F,NaN,0,0,N,0,N,0,N,0,0,0,0,8.0,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,8,0,0,0,0,0,0,0,4
4,ANA201804020,CLE,1,1,1,0,2,CCT,0,0,troum001,R,troum001,R,clevm001,R,clevm001,R,gomey001,alony001,kipnj001,ramij003,lindf001,naqut001,zimmb001,chisl001,NaN,NaN,NaN,K,F,F,8,2,3,T,T,0,F,F,1,F,F,0,F,F,0,NaN,F,F,NaN,0,0,N,0,N,0,N,0,0,0,0,2.0,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,2,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185134,WAS201809260,MIA,7,0,2,1,2,FFBX,3,9,dietd001,L,dietd001,L,rodrj005,R,rodrj005,R,kiebs001,reynm001,sanca007,renda001,turnt001,sotoj001,roblv001,harpb003,NaN,NaN,NaN,53/G,F,F,3,3,2,T,T,0,F,F,1,F,F,0,F,F,5,G,F,F,NaN,0,0,N,0,N,0,N,0,0,0,0,53.0,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,3,0,0,5,0,0,0,0,66
185135,WAS201809260,MIA,7,1,0,3,1,.BBFBB,3,9,kiebs001,R,kiebs001,R,herne002,R,herne002,R,wallc001,dietd001,bostc001,andeb006,rojam002,orter001,brinl001,obrip002,NaN,NaN,NaN,W,T,F,2,8,14,T,F,0,F,F,0,F,F,0,F,F,0,NaN,F,F,NaN,0,0,N,0,N,0,N,1,0,0,0,NaN,NaN,NaN,NaN,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,F,F,F,F,F,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,67
185136,WAS201809260,MIA,7,1,0,0,2,.SFS,3,9,eatoa002,L,eatoa002,L,herne002,R,herne002,R,wallc001,dietd001,bostc001,andeb006,rojam002,orter001,brinl001,obrip002,

# Add columns

In [16]:
data['HOME_TEAM_ID'] = data.apply(lambda row: row.GAME_ID[:3], axis=1)

# START_BASES_CD, END_BASES_CD = 4 * (3루) + 2 * (2루) + 1 * (1루)
data['START_BASES_CD'] = 1 * data['BASE1_RUN_ID'].notna() + \
                         2 * data['BASE2_RUN_ID'].notna() + \
                         4 * data['BASE3_RUN_ID'].notna()

data['END_BASES_CD'] = 1 * (data[['BAT_DEST_ID', 'RUN1_DEST_ID', 'RUN2_DEST_ID', 'RUN3_DEST_ID']] == 1).apply(
    lambda x: sum(x) >= 1, axis=1) + \
                       2 * (data[['BAT_DEST_ID', 'RUN1_DEST_ID', 'RUN2_DEST_ID', 'RUN3_DEST_ID']] == 2).apply(
    lambda x: sum(x) >= 1, axis=1) + \
                       4 * (data[['BAT_DEST_ID', 'RUN1_DEST_ID', 'RUN2_DEST_ID', 'RUN3_DEST_ID']] == 3).apply(
    lambda x: sum(x) >= 1, axis=1)

data['EVENT_RUNS_CT'] = (data[['BAT_DEST_ID', 'RUN1_DEST_ID', 'RUN2_DEST_ID', 'RUN3_DEST_ID']] >= 4).apply(
    lambda x: sum(x), axis=1)

data['END_OUTS_CT'] = data['OUTS_CT'] + data['EVENT_OUTS_CT']

# INN_NEW_FL: OUTS_CT == 0 and 주자 == 0, INN_END_FL: END_OUTS_CT == 3
data['INN_NEW_FL'] = (data['OUTS_CT'] == 0) & (data['START_BASES_CD'] == 0)
data['INN_END_FL'] = data['END_OUTS_CT'] == 3

In [6]:
# 초반, 후반 데이터 나누기.
n_games = sum(data.GAME_NEW_FL == 'T')  # 게임 수
train = data[data.GAME_ID.isin(data.GAME_ID.unique()[:n_games//2])]  # 전반기
test = data[data.GAME_ID.isin(data.GAME_ID.unique()[n_games//2:])]  # 후반기

# Transition matrix 구하기

In [7]:
# state_action_cnt: state -> action 경우의 수
state_action_cnt = train.groupby(['OUTS_CT', 'START_BASES_CD'])['EVENT_CD'].value_counts()
state_action_cnt

OUTS_CT  START_BASES_CD  EVENT_CD
0        0               2           10604
                         3            4988
                         20           3268
                         14           1644
                         21           1035
                                     ...  
2        7               23             39
                         16             16
                         18             11
                         22              7
                         19              1
Name: EVENT_CD, Length: 262, dtype: int64

In [8]:
# result_cnt: (state, action) -> state 경우의 수
result_cnt = pd.pivot_table(train, values='EVENT_ID', index=['OUTS_CT', 'START_BASES_CD', 'EVENT_CD'],
                            columns=['END_OUTS_CT', 'END_BASES_CD'], aggfunc='count').fillna(0.0).astype(int)
result_cnt

END_OUTS_CT                      0                              1              \
END_BASES_CD                     0     1  2  3  4  5  6  7      0  1  2  3  4   
OUTS_CT START_BASES_CD EVENT_CD                                                 
0       0              2         0     8  0  0  0  0  0  0  10596  0  0  0  0   
                       3         0    15  0  0  0  0  0  0   4973  0  0  0  0   
                       14        0  1644  0  0  0  0  0  0      0  0  0  0  0   
                       16        0   236  0  0  0  0  0  0      0  0  0  0  0   
                       17        0     4  0  0  0  0  0  0      0  0  0  0  0   
...                             ..   ... .. .. .. .. .. ..    ... .. .. .. ..   
2       7              19        0     0  0  0  0  0  0  0      0  0  0  0  0   
                       20        0     0  0  0  0  0  0  0      0  0  0  0  0   
                       21        0     0  0  0  0  0  0  0      0  0  0  0  0   
                       22        0     0  0  0  0  0  0  0      0  0  0  0  0   
                       23        0     0  0  0  0  0  0  0      0  0  0  0  0   

END_OUTS_CT                                2                            3     \
END_BASES_CD                     5  6  7   0  1   2   3  4   5   6   7  0  1   
OUTS_CT START_BASES_CD EVENT_CD                                                
0       0              2         0  0  0   0  0   0   0  0   0   0   0  0  0   
                       3         0  0  0   0  0   0   0  0   0   0   0  0  0   
                       14        0  0  0   0  0   0   0  0   0   0   0  0  0   
                       16        0  0  0   0  0   0   0  0   0   0   0  0  0   
                       17        0  0  0   0  0   0   0  0   0   0   0  0  0   
...                             .. .. ..  .. ..  ..  .. ..  ..  ..  .. .. ..   
2       7              19        0  0  0   0  0   0   0  0   0   0   1  0  0   
                       20        0  0  0   0  1   2  36  0  35   9  30  0  5   
                       21        0  0  0   0  0  20   0  4   0  20   0  1  0   
                       22        0  0  0   0  0   0   0  7   0   0   0  0  0   
                       23        0  0  0  39  0   0   0  0   0   0   0  0  0   

END_OUTS_CT                                        
END_BASES_CD                     2  3  4  5  6  7  
OUTS_CT START_BASES_CD EVENT_CD                    
0       0              2         0  0  0  0  0  0  
                       3         0  0  0  0  0  0  
                       14        0  0  0  0  0  0  
                       16        0  0  0  0  0  0  
                       17        0  0  0  0  0  0  
...                             .. .. .. .. .. ..  
2       7              19        0  0  0  0  0  0  
                       20        0  1  0  0  0  0  
                       21        1  0  0  0  0  0  
                       22        0  0  0  0  0  0  
                       23        0  0  0  0  0  0  

[262 rows x 32 columns]

In [9]:
# runs_cnt: state -> state 총 득점
runs_cnt = pd.pivot_table(train, values='EVENT_RUNS_CT', index=['OUTS_CT', 'START_BASES_CD'],
                          columns=['END_OUTS_CT', 'END_BASES_CD'], aggfunc='sum').fillna(0.0).astype(int)
runs_cnt

END_OUTS_CT               0                                1                 \
END_BASES_CD              0   1   2   3   4   5   6   7    0    1    2    3   
OUTS_CT START_BASES_CD                                                        
0       0               765   0   0   0   0   0   0   0    0    0    0    0   
        1               316   0  64   0  29   0   0   0    2    0    0    0   
        2                64  78  75   0   2   0   0   0    4    0    0    0   
        3               102   0  42  33  16  28  42   0    0    2    5    0   
        4                10  30  15   0   3   0   0   0   53    0    0    0   
        5                51   0   4  61   4  16  14   0    0   78   24    0   
        6                30  34  32   3   0  30   1   0    0    0   25    0   
        7                44   0   6  18   6  16  46  66    3    2    0   27   
1       0                 0   0   0   0   0   0   0   0  530    0    0    0   
        1                 0   0   0   0   0   0   0   0  362    4   88    0   
        2                 0   0   0   0   0   0   0   0  146  154  128    0   
        3                 0   0   0   0   0   0   0   0  186    0   66   98   
        4                 0   0   0   0   0   0   0   0   46  167   38    0   
        5                 0   0   0   0   0   0   0   0  102    2   24  137   
        6                 0   0   0   0   0   0   0   0   48   80   68    6   
        7                 0   0   0   0   0   0   0   0  108    3   45   66   
2       0                 0   0   0   0   0   0   0   0    0    0    0    0   
        1                 0   0   0   0   0   0   0   0    0    0    0    0   
        2                 0   0   0   0   0   0   0   0    0    0    0    0   
        3                 0   0   0   0   0   0   0   0    0    0    0    0   
        4                 0   0   0   0   0   0   0   0    0    0    0    0   
        5                 0   0   0   0   0   0   0   0    0    0    0    0   
        6                 0   0   0   0   0   0   0   0    0    0    0    0   
        7                 0   0   0   0   0   0   0   0    0    0    0    0   

END_OUTS_CT                                2                               \
END_BASES_CD             4   5   6    7    0    1    2    3    4    5   6   
OUTS_CT START_BASES_CD                                                      
0       0                0   0   0    0    0    0    0    0    0    0   0   
        1                0   0   0    0    0    0    0    0    0    0   0   
        2                0   0   0    0    0    0    0    0    0    0   0   
        3                1   0   0    0    0    0    0    0    0    0   0   
        4                0   0   0    0    0    0    0    0    0    0   0   
        5                4   0   0    0   21    0    0    0    0    0   0   
        6               43   0   0    0    0    0    0    0    0    0   0   
        7                0  28  10    0    0    0    1    0   17    0   0   
1       0                0   0   0    0    0    0    0    0    0    0   0   
        1               52   0   0    0    4    0    0    0    0    0   0   
        2               11   0   0    0   10    0    0    0    0    0   0   
        3               32  63  91    0    2    9    7    0    4    0   0   
        4               13   0   0    0  185    0    0    0    0    0   0   
        5               14  44  35    0    4  147   69    0    3    0   0   
        6               14  83   6    0    2    4   68    0  109    0   0   
        7               18  42  66  136    6    2    4   47    2   68  36   
2       0                0   0   0    0  382    0    0    0    0    0   0   
        1                0   0   0    0  378    3  142    0   41    0   0   
        2                0   0   0    0  178  268  184    0   30    0   0   
        3                0   0   0    0  183    2  154  116   46  122  64   
        4                0   0   0    0   78  161   61    0    4    0   0   
        5         

In [10]:
# trans_cnt: state -> state 경우의 수
trans_cnt = pd.pivot_table(train, values='EVENT_RUNS_CT', index=['OUTS_CT', 'START_BASES_CD'],
                           columns=['END_OUTS_CT', 'END_BASES_CD'], aggfunc='count').fillna(0.0).astype(int)
trans_cnt

END_OUTS_CT               0                                            1  \
END_BASES_CD              0     1     2     3    4    5    6    7      0   
OUTS_CT START_BASES_CD                                                     
0       0               765  5258  1078     0  110    0    0    0  15601   
        1               158     0    64  1036   29  214  172    0      2   
        2                32    78    75   160    2  135    6    0      4   
        3                34     0    21    33    8   28   42  209      0   
        4                 5    30    15     0    3   32    0    0     53   
        5                17     0     2    61    2   16   14   30      0   
        6                10    17    16     3    0   30    1   37      0   
        7                11     0     2     9    2    8   23   66      1   
1       0                 0     0     0     0    0    0    0    0    530   
        1                 0     0     0     0    0    0    0    0    181   
        2                 0     0     0     0    0    0    0    0     73   
        3                 0     0     0     0    0    0    0    0     62   
        4                 0     0     0     0    0    0    0    0     23   
        5                 0     0     0     0    0    0    0    0     34   
        6                 0     0     0     0    0    0    0    0     16   
        7                 0     0     0     0    0    0    0    0     27   
2       0                 0     0     0     0    0    0    0    0      0   
        1                 0     0     0     0    0    0    0    0      0   
        2                 0     0     0     0    0    0    0    0      0   
        3                 0     0     0     0    0    0    0    0      0   
        4                 0     0     0     0    0    0    0    0      0   
        5                 0     0     0     0    0    0    0    0      0   
        6                 0     0     0     0    0    0    0    0      0   
        7                 0     0     0     0    0    0    0    0      0   

END_OUTS_CT                                                      2        \
END_BASES_CD               1    2     3    4    5    6    7      0     1   
OUTS_CT START_BASES_CD                                                     
0       0                  0    0     0    0    0    0    0      0     0   
        1               2343  396     0   11    0    0    0    537     0   
        2                 20  638     0  372    0    0    0     14     0   
        3                  2    5   473    1  127  112    0      0    15   
        4                  3    0     0   87    0    0    0      0     0   
        5                 78   24    15    4  131   13    0     21     0   
        6                  0   25     4   43    3  130    0      0     0   
        7                  1    0    27    0   28   10  118      0     0   
1       0               3883  749     0   93    0    0    0  11439     0   
        1                  4   88  1203   52  265  228    0      4  2892   
        2                154  128   326   11  178    9    0     10    21   
        3                  0   33    99   16   63   91  347      1     9   
        4                167   38     0   13  145    0    0    185    26   
        5                  1   12   137    7   44   35   99      2   147   
        6                 40   34     6    7   83    6  155      1     4   
        7                  1   15    33    6   21   33  136      2     1   
2       0                  0    0     0    0    0    0    0    382  3163   
        1                  0    0     0    0    0    0    0    189     3   
        2                  0    0     0    0    0    0    0     89   268   
        3                  0    0     0    0    0    0    0     61     1   
        4                  0    0     0    0    0    0    0     39   161   
        5                  0    0     0    0    0    0    0     32     0   
        6                  0    0     0    0   

In [11]:
# R: state -> state 득점
R = (runs_cnt / trans_cnt).fillna(0.0)
R

END_OUTS_CT               0                                       1            \
END_BASES_CD              0    1    2    3    4    5    6    7    0    1    2   
OUTS_CT START_BASES_CD                                                          
0       0               1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        1               2.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0   
        2               2.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0   
        3               3.0  0.0  2.0  1.0  2.0  1.0  1.0  0.0  0.0  1.0  1.0   
        4               2.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0   
        5               3.0  0.0  2.0  1.0  2.0  1.0  1.0  0.0  0.0  1.0  1.0   
        6               3.0  2.0  2.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0  1.0   
        7               4.0  0.0  3.0  2.0  3.0  2.0  2.0  1.0  3.0  2.0  0.0   
1       0               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
        1               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  1.0  1.0   
        2               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  1.0  1.0   
        3               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  2.0   
        4               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  1.0  1.0   
        5               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  2.0  2.0   
        6               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  2.0  2.0   
        7               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  3.0  3.0   
2       0               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        1               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        4               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        5               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        6               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        7               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

END_OUTS_CT                                             2                      \
END_BASES_CD                   3    4    5    6    7    0    1    2    3    4   
OUTS_CT START_BASES_CD                                                          
0       0               0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        1               0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2               0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3               0.000000  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        4               0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        5               0.000000  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
        6               0.000000  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        7               1.000000  0.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0   
1       0               0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        1               0.000000  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
        2               0.000000  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
        3               0.989899  2.0  1.0  1.0  0.0  2.0  1.0  1.0  0.0  1.0   
        4               0.000000  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
        5               1.000000  2.0  1.0  1.0  0.0  2.0  1.0  1.0  0.0  1.0   
        6               1.000000  2.0  1.0  1.0  0.0  2.0  1.0  1.0  0.0  1.0   
        7               2.000000  3.0  2.0  2.0  1.0  3.0  2.0  2.0  1.0  2.0   
2       0               0.000000  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
        1               0.000000  0.0  0.0  0.0  0.0  2.0  1.0  1.0  0.0  1.0   
        2               0.000000  0.0  0.0  0.0  0.0  2.0  1.0  1.0  0.0  1.0   
        3               0.0000

In [12]:
# TA: (state, action) -> state 확률
TA = result_cnt.apply(lambda x: x / state_action_cnt[result_cnt.index])
TA

END_OUTS_CT                        0                                          \
END_BASES_CD                       0         1    2    3    4    5    6    7   
OUTS_CT START_BASES_CD EVENT_CD                                                
0       0              2         0.0  0.000754  0.0  0.0  0.0  0.0  0.0  0.0   
                       3         0.0  0.003007  0.0  0.0  0.0  0.0  0.0  0.0   
                       14        0.0  1.000000  0.0  0.0  0.0  0.0  0.0  0.0   
                       16        0.0  1.000000  0.0  0.0  0.0  0.0  0.0  0.0   
                       17        0.0  1.000000  0.0  0.0  0.0  0.0  0.0  0.0   
...                              ...       ...  ...  ...  ...  ...  ...  ...   
2       7              19        0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
                       20        0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
                       21        0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
                       22        0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
                       23        0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   

END_OUTS_CT                             1                                     \
END_BASES_CD                            0    1    2    3    4    5    6    7   
OUTS_CT START_BASES_CD EVENT_CD                                                
0       0              2         0.999246  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                       3         0.996993  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                       14        0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                       16        0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                       17        0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                                   ...  ...  ...  ...  ...  ...  ...  ...   
2       7              19        0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                       20        0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                       21        0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                       22        0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                       23        0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

END_OUTS_CT                        2                                          \
END_BASES_CD                       0         1         2         3         4   
OUTS_CT START_BASES_CD EVENT_CD                                                
0       0              2         0.0  0.000000  0.000000  0.000000  0.000000   
                       3         0.0  0.000000  0.000000  0.000000  0.000000   
                       14        0.0  0.000000  0.000000  0.000000  0.000000   
                       16        0.0  0.000000  0.000000  0.000000  0.000000   
                       17        0.0  0.000000  0.000000  0.000000  0.000000   
...                              ...       ...       ...       ...       ...   
2       7              19        0.0  0.000000  0.000000  0.000000  0.000000   
                       20        0.0  0.008403  0.016807  0.302521  0.000000   
                       21        0.0  0.000000  0.434783  0.000000  0.086957   
                       22        0.0  0.000000  0.000000  0.000000  1.000000   
                       23        1.0  0.000000  0.000000  0.000000  0.000000   

END_OUTS_CT                                                           3  \
END_BASES_CD                            5         6         7         0   
OUTS_CT START_BASES_CD EVENT_CD                                           
0       0              2         0.000000  0.000000  0.000000  0.000000   
                       3         0.000000  0.000000  0.000000  0.000000   
                       14        0.000000  0.000000  0.000000  0.000000   
                       16        0.000000  0.000000  0.000000  0.000000   
                       17        0.000000  0.000000  0.000000  0.000000   
...                                  

In [13]:
# state_cnt: 어떤 state의 총 횟수
state_cnt = trans_cnt.apply(lambda x: sum(x), axis=1)
state_cnt

OUTS_CT  START_BASES_CD
0        0                 22812
         1                  4962
         2                  1536
         3                  1233
         4                   228
         5                   437
         6                   320
         7                   331
1        0                 16694
         1                  6126
         2                  2457
         3                  2184
         4                   897
         5                  1016
         6                   812
         7                   827
2        0                 13242
         1                  5981
         2                  3345
         3                  2689
         4                  1345
         5                  1194
         6                   965
         7                  1078
dtype: int64

In [14]:
# T: state -> state 확률
T = trans_cnt.apply(lambda x: x / state_cnt[x.index])
T

END_OUTS_CT                    0                                          \
END_BASES_CD                   0         1         2         3         4   
OUTS_CT START_BASES_CD                                                     
0       0               0.033535  0.230493  0.047256  0.000000  0.004822   
        1               0.031842  0.000000  0.012898  0.208787  0.005844   
        2               0.020833  0.050781  0.048828  0.104167  0.001302   
        3               0.027575  0.000000  0.017032  0.026764  0.006488   
        4               0.021930  0.131579  0.065789  0.000000  0.013158   
        5               0.038902  0.000000  0.004577  0.139588  0.004577   
        6               0.031250  0.053125  0.050000  0.009375  0.000000   
        7               0.033233  0.000000  0.006042  0.027190  0.006042   
1       0               0.000000  0.000000  0.000000  0.000000  0.000000   
        1               0.000000  0.000000  0.000000  0.000000  0.000000   
        2               0.000000  0.000000  0.000000  0.000000  0.000000   
        3               0.000000  0.000000  0.000000  0.000000  0.000000   
        4               0.000000  0.000000  0.000000  0.000000  0.000000   
        5               0.000000  0.000000  0.000000  0.000000  0.000000   
        6               0.000000  0.000000  0.000000  0.000000  0.000000   
        7               0.000000  0.000000  0.000000  0.000000  0.000000   
2       0               0.000000  0.000000  0.000000  0.000000  0.000000   
        1               0.000000  0.000000  0.000000  0.000000  0.000000   
        2               0.000000  0.000000  0.000000  0.000000  0.000000   
        3               0.000000  0.000000  0.000000  0.000000  0.000000   
        4               0.000000  0.000000  0.000000  0.000000  0.000000   
        5               0.000000  0.000000  0.000000  0.000000  0.000000   
        6               0.000000  0.000000  0.000000  0.000000  0.000000   
        7               0.000000  0.000000  0.000000  0.000000  0.000000   

END_OUTS_CT                                                  1            \
END_BASES_CD                   5         6         7         0         1   
OUTS_CT START_BASES_CD                                                     
0       0               0.000000  0.000000  0.000000  0.683894  0.000000   
        1               0.043128  0.034663  0.000000  0.000403  0.472189   
        2               0.087891  0.003906  0.000000  0.002604  0.013021   
        3               0.022709  0.034063  0.169505  0.000000  0.001622   
        4               0.140351  0.000000  0.000000  0.232456  0.013158   
        5               0.036613  0.032037  0.068650  0.000000  0.178490   
        6               0.093750  0.003125  0.115625  0.000000  0.000000   
        7               0.024169  0.069486  0.199396  0.003021  0.003021   
1       0               0.000000  0.000000  0.000000  0.031748  0.232599   
        1               0.000000  0.000000  0.000000  0.029546  0.000653   
        2               0.000000  0.000000  0.000000  0.029711  0.062678   
        3               0.000000  0.000000  0.000000  0.028388  0.000000   
        4               0.000000  0.000000  0.000000  0.025641  0.186176   
        5               0.000000  0.000000  0.000000  0.033465  0.000984   
        6               0.000000  0.000000  0.000000  0.019704  0.049261   
        7               0.000000  0.000000  0.000000  0.032648  0.001209   
2       0               0.000000  0.000000  0.000000  0.000000  0.000000   
        1               0.000000  0.000000  0.000000  0.000000  0.000000   
        2               0.000000  0.000000  0.000000  0.000000  0.000000   
        3               0.000000  0.000000  0.000000  0.000000  0.000000   
        4               0.000000  0.000000  0.000000  0.000000  0.000000   
        5               0.000000  0.000000  0.000000  0.000000  0.000000   
        6               0.000000  0.000000  0.0

# 타자, 투수 policy

In [17]:
min_pa = 100  # 최소 타석 수.

In [18]:
# 초기화
bat = {}
for bat_id in train.BAT_ID.unique():
    bat[bat_id] = {key: 0 for key in pa_events + ['pa']}
    
# 각 타자의 event 개수 세기.
for index, row in train.iterrows():
    bat_id = row['BAT_ID']
    event = row['EVENT_CD']
    bat[bat_id]['pa'] += 1
    bat[bat_id][event] += 1
    
# 평균 타자 기록 (pa < min_pa)
bat['average'] = {key: 0 for key in pa_events + ['pa']}
for bat_id in bat:
    bat['average']['pa'] += bat[bat_id]['pa']
    for event in pa_events:
        bat['average'][event] += bat[bat_id][event]

In [19]:
# 초기화
pit = {}
for pit_id in train.PIT_ID.unique():
    pit[pit_id] = {key: 0 for key in pa_events + ['pa']}
    
# 각 투수의 event 개수 세기.
for index, row in train.iterrows():
    pit_id = row['PIT_ID']
    event = row['EVENT_CD']
    pit[pit_id]['pa'] += 1
    pit[pit_id][event] += 1
    
# 평균 투수 기록 (pa < min_pa)
pit['average'] = {key: 0 for key in pa_events + ['pa']}
for pit_id in pit:
    pit['average']['pa'] += pit[pit_id]['pa']
    for event in pa_events:
        pit['average'][event] += pit[pit_id][event]

# 각 타석에서 각 event가 일어날 확률을 학습

In [21]:
# event_dict 초기화.
event_dict = {}
for event in pa_events:
    event_dict['BAT_'+str(event)] = []
    event_dict['PIT_'+str(event)] = []
    
other_features = {'EVENT_CD'}
for feature in other_features:
    event_dict[feature] = []

# 각 타석에서 타자 policy와 투수 policy를 가져옴.
for index, row in train.iterrows():
    bat_id = row['BAT_ID']
    pit_id = row['PIT_ID']
    
    for event in pa_events:
        if bat[bat_id]['pa'] >= min_pa:
            event_dict['BAT_'+str(event)].append(bat[bat_id][event] / bat[bat_id]['pa'])
        else:
            event_dict['BAT_'+str(event)].append(bat['average'][event] / bat['average']['pa'])
        
        if pit[pit_id]['pa'] >= min_pa:
            event_dict['PIT_'+str(event)].append(pit[pit_id][event] / pit[pit_id]['pa'])
        else:
            event_dict['PIT_'+str(event)].append(pit['average'][event] / pit['average']['pa'])
            
    for feature in other_features:
        event_dict[feature].append(row[feature])

event_df = pd.DataFrame(event_dict)
event_df

,BAT_2,PIT_2,BAT_3,PIT_3,BAT_14,PIT_14,BAT_15,PIT_15,BAT_16,PIT_16,BAT_17,PIT_17,BAT_18,PIT_18,BAT_19,PIT_19,BAT_20,PIT_20,BAT_21,PIT_21,BAT_22,PIT_22,BAT_23,PIT_23,EVENT_CD
0,0.508532,0.450378,0.139932,0.220190,0.083618,0.079430,0.008532,0.005253,0.011945,0.010549,0.000000,0.000291,0.005119,0.008079,0.000000,0.002837,0.145051,0.143122,0.049488,0.044676,0.001706,0.004789,0.046075,0.030406,2
1,0.476496,0.450378,0.183761,0.220190,0.102564,0.079430,0.002137,0.005253,0.010684,0.010549,0.000000,0.000291,0.010684,0.008079,0.002137,0.002837,0.121795,0.143122,0.053419,0.044676,0.002137,0.004789,0.034188,0.030406,2
2,0.478976,0.450378,0.117002,0.220190,0.146252,0.079430,0.021938,0.005253,0.007313,0.010549,0.000000,0.000291,0.009141,0.008079,0.001828,0.002837,0.106033,0.143122,0.051188,0.044676,0.005484,0.004789,0.054845,0.030406,2
3,0.518293,0.449695,0.201220,0.257622,0.085366,0.076220,0.000000,0.000000,0.024390,0.004573,0.000000,0.001524,0.012195,0.004573,0.000000,0.001524,0.091463,0.128049,0.048780,0.044207,0.000000,0.004573,0.018293,0.027439,2
4,0.330357,0.449695,0.214286,0.257622,0.176339,0.076220,0.037946,0.000000,0.015625,0.004573,0.002232,0.001524,0.002232,0.004573,0.000000,0.001524,0.131696,0.128049,0.024554,0.044207,0.006696,0.004573,0.058036,0.027439,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92706,0.394904,0.422857,0.248408,0.257143,0.095541,0.120000,0.006369,0.017143,0.019108,0.022857,0.000000,0.000000,0.000000,0.011429,0.000000,0.005714,0.146497,0.114286,0.057325,0.017143,0.006369,0.000000,0.025478,0.011429,2
92707,0.455357,0.422857,0.169643,0.257143,0.071429,0.120000,0.008929,0.017143,0.017857,0.022857,0.000000,0.000000,0.000000,0.011429,0.008929,0.005714,0.160714,0.114286,0.080357,0.017143,0.000000,0.000000,0.026786,0.011429,2
92708,0.432718,0.464115,0.242744,0.267943,0.050132,0.066986,0.005277,0.004785,0.034301,0.014354,0.000000,0.000000,0.007916,0.009569,0.002639,0.000000,0.153034,0.081340,0.044855,0.052632,0.005277,0.014354,0.021108,0.023923,3
92709,0.447598,0.464115,0.183406,0.267943,0.085153,0.066986,0.002183,0.004785,0.026201,0.014354,0.000000,0.000000,0.015284,0.009569,0.004367,0.000000,0.163755,0.081340,0.048035,0.052632,0.006550,0.014354,0.017467,0.023923,21


In [22]:
X = event_df.drop('EVENT_CD', axis=1)
y = event_df['EVENT_CD']

In [30]:
(y.value_counts() / y.value_counts().sum()).sort_index()

2     0.450378
3     0.220190
14    0.079430
15    0.005253
16    0.010549
17    0.000291
18    0.008079
19    0.002837
20    0.143122
21    0.044676
22    0.004789
23    0.030406
Name: EVENT_CD, dtype: float64

In [23]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(n_jobs=-1)
model.fit(X, y)
pd.DataFrame(model.predict_proba(X), columns=pa_events).mean()

2     0.451412
3     0.219939
14    0.079316
15    0.005386
16    0.010191
17    0.000225
18    0.008045
19    0.003016
20    0.142734
21    0.044866
22    0.004647
23    0.030224
dtype: float64

In [24]:
model.coef_

array([[ 3.59370394e+00,  2.87738016e+00,  6.94752091e-01,
         9.56630239e-01, -9.66620613e-01, -1.07801224e+00,
        -4.09369611e-01, -1.67952703e-01, -3.49657955e-01,
        -3.64171626e-01, -9.63339485e-03, -1.89830490e-02,
        -1.67068831e-01, -1.77366406e-01, -6.12486869e-02,
        -9.81974476e-02,  6.24799555e-02, -4.24477921e-01,
        -1.08693348e+00, -8.08013905e-01, -1.65091734e-01,
        -4.06366777e-02, -1.06865311e+00, -5.89539858e-01],
       [ 9.71790743e-01,  4.85073252e-01,  4.60861576e+00,
         4.76869660e+00, -1.46392878e+00, -1.34681267e+00,
        -2.83422456e-01, -2.02908459e-01, -3.45058510e-01,
        -3.21543186e-01, -7.51020811e-03,  1.38839509e-02,
        -4.80729424e-02, -1.35436693e-01, -1.48262463e-02,
        -1.61473054e-01, -7.12768068e-01, -1.16133826e+00,
        -1.12092681e+00, -9.13958700e-01, -6.57246279e-02,
        -1.94859355e-01, -1.39809032e+00, -7.09245897e-01],
       [-7.62703069e-01, -1.20537173e+00, -1.09899924e

In [26]:
model.intercept_

array([ 0.26504983,  0.40115495,  2.20508259, -0.32800732, -0.10880741,
       -3.11302513, -0.62778262, -1.29298468,  0.95977704,  1.23354365,
       -0.90277113,  1.30877023])

# 각 팀별 평균 수비력

In [ ]:
away_event_cnt = train.groupby('AWAY_TEAM_ID')['EVENT_CD'].value_counts()
home_event_cnt = train.groupby('HOME_TEAM_ID')['EVENT_CD'].value_counts()

team_info = {}
for team, event in away_event_cnt.index:
    if team not in team_info:
        team_info[team] = {event: 0 for event in pa_events + ['pa']}
    team_info[team][event] += away_event_cnt[(team, event)]
    team_info[team]['pa'] += away_event_cnt[(team, event)]
for team, event in home_event_cnt.index:
    if team not in team_info:
        team_info[team] = {event: 0 for event in pa_events + ['pa']}
    team_info[team][event] += home_event_cnt[(team, event)]
    team_info[team]['pa'] += home_event_cnt[(team, event)]

In [ ]:
tmp = {2: 0.45, 3: 0.22}
sum(tmp.values())

In [ ]:
# (action, proba)를 dictionary 형태로 받아서 random action을 선택하는 함수
def choose_rand_action(probas):
    probas = dict(probas)  # {2: 0.45, 3: 0.22, ...}
    proba_sum = sum(probas.values())
    choice = random.uniform(0, proba_sum)
    accu = 0
    for action, proba in probas.items():
        accu += proba
        if choice < accu:
            return action
    return action

In [ ]:
def get_policy(bat_id, pit_id, state, pit_team, sec_pit=False):
    features = []
    for event in pa_events:
        if bat_id in bat and bat[bat_id]['pa'] >= min_pa:
            features.append(bat[bat_id][event] / bat[bat_id]['pa'])
        else:
            features.append(bat['average'][event] / bat['average']['pa'])
        
        if not sec_pit:  # 선발 투수인 경우.
            if pit_id in pit and pit[pit_id]['pa'] >= min_pa:
                features.append(pit[pit_id][event] / pit[pit_id]['pa'])
            else:
                features.append(pit['average'][event] / pit['average']['pa'])
        else:  # 선발 투수가 아닌 경우.
            if pit_id in pit:
                proba = (team_info[pit_team][event] - pit[pit_id][event]) / (team_info[pit_team]['pa'] - pit[pit_id]['pa'])
            else:
                proba = team_info[pit_team][event] / team_info[pit_team]['pa']
            features.append(proba)
            
    features = features + state
    
    probas = model.predict_proba([features])
    return {event: probas[0, index] for index, event in enumerate(pa_events)}

# Simulation

In [ ]:
n_simuls = 250
n_procs = 25

In [ ]:
# X: input features of the model. shape=(f, ).
# W: model weights. shape=(m, f).
# B: model bias. shape=(m, ).
# T: (state, action) -> state transition matrix. shape=(m, k).
# V: value functions of the next states. shape=(k, ).
# U: state -> state transition matrix. shape=(k, k).
# R: shape=(k, ).
def get_gradient(X, W, B, T, U, R, V, v_truth):
    X1 = np.concatenate((X, [1]))  # shape=(f+1, )
    Z = np.dot(np.concatenate((W, B.reshape(-1, 1)), axis=1), X1)  # shape=(m, )
    S = 1 / (1 + np.exp(-Z))  # shape=(m, )
    Q = (V + R).reshape(1, -1) * T  # shape=(m, k)
    v_prime = (V + R).dot(U)  # shape=(1, )
    A = Q * (S * (1 - S)).reshape(-1, 1)  # shape=(m, k)
    B = np.repeat(X1.reshape(1, -1), A.shape[1], axis=0)  # shape=(k, f+1)
    return (v_prime - v_truth) * np.dot(A, B)  # shape=(m, f+1)

In [ ]:
def update_weights(df, model):
    df['INN_END_FL']

In [ ]:
def simulation(away_scores, home_scores, case_num):
    state = {
        'INN_CT': 1,
        'BAT_HOME_ID': 0,
        'OUTS_CT': 0,
        'BASE1_RUN_FL': False,
        'BASE2_RUN_FL': False,
        'BASE3_RUN_FL': False,
        'AWAY_BAT_CD': 1,
        'HOME_BAT_CD': 1,
        'AWAY_SCORE_CT': 0,
        'HOME_SCORE_CT': 0,
        'END_FL': False
    }

    while state['END_FL'] == False:  # Until the game ends.
        this_state = [state['OUTS_CT'], state['BASE1_RUN_FL'], state['BASE2_RUN_FL'], state['BASE3_RUN_FL']]
        
        # policy(action 선택 확률) 구하기
        if state['BAT_HOME_ID'] == 0:  # away team batting
            policy = get_policy(away_batters[state['AWAY_BAT_CD']-1], home_pitcher,
                                this_state,
                                home_team, state['INN_CT'] >= 6)
        else:                          # home team batting
            policy = get_policy(home_batters[state['HOME_BAT_CD']-1], away_pitcher,
                                this_state,
                                away_team, state['INN_CT'] >= 6)
            
        # For updating the model weights.
        get_gradient()

        # action 구하기
        action = choose_rand_action(policy)
        state_action = (state['OUTS_CT'], state['BASE1_RUN_FL'], state['BASE2_RUN_FL'], state['BASE3_RUN_FL'], action)
        if state_action not in T.index:
            continue

        # 다음 state 구하기
        trans_probas = T.loc[state_action]
        next_state = choose_rand_action(trans_probas)
        state['OUTS_CT'], state['BASE1_RUN_FL'], state['BASE2_RUN_FL'], state['BASE3_RUN_FL'] = next_state
        if state['OUTS_CT'] == 3:
            state['OUTS_CT'] = 0
            state['BASE1_RUN_FL'], state['BASE2_RUN_FL'], state['BASE3_RUN_FL'] = False, False, False
            inning_end = True
        else:
            inning_end = False

        # 득점 반영
        runs_scored += R.at[this_state, next_state]
        if state['BAT_HOME_ID'] == 0:  # away team batting
            state['AWAY_BAT_CD'] = state['AWAY_BAT_CD'] % 9 + 1
            state['AWAY_SCORE_CT'] += runs_scored
        else:                          # home team batting
            state['HOME_BAT_CD'] = state['HOME_BAT_CD'] % 9 + 1
            state['HOME_SCORE_CT'] += runs_scored

        # 이닝이 끝날 경우
        if inning_end:
            # x회 말일 경우 inning += 1
            if state['BAT_HOME_ID'] == 1:
                state['INN_CT'] += 1

            # 투타 전환
            state['BAT_HOME_ID'] = 1 - state['BAT_HOME_ID']

            # 9회까지 했을 때 경기 종료
            if state['INN_CT'] > 9:
                state['END_FL'] = True

    away_scores[case_num] = state['AWAY_SCORE_CT']
    home_scores[case_num] = state['HOME_SCORE_CT']

In [ ]:
def proc_func(away_scores, home_scores, begin, end):
    for case_num in range(begin, end):
        simulation(away_scores, home_scores, case_num)

In [ ]:
f = open("log.txt", "w")  # 로그 파일 열기

# 게임이 시작하는 indices.
start_indices = list(test.index[test['GAME_NEW_FL'] == 'T']) + [test.index[-1] + 1]

trues, total = 0, 0  # total: 무승부를 제외한 모든 경기.

for i in trange(len(start_indices)-1):
    this_game = test.loc[start_indices[i]:start_indices[i+1]-1]
    
    away_team = this_game.iloc[0]['AWAY_TEAM_ID']
    home_team = this_game.iloc[0]['HOME_TEAM_ID']
    
    away_batters = list(this_game['BAT_ID'][this_game['BAT_HOME_ID'] == 0].head(9))
    home_batters = list(this_game['BAT_ID'][this_game['BAT_HOME_ID'] == 1].head(9))

    home_pitcher = this_game['PIT_ID'][this_game['BAT_HOME_ID'] == 0].iloc[0]
    away_pitcher = this_game['PIT_ID'][this_game['BAT_HOME_ID'] == 1].iloc[0]
    
    with Manager() as manager:
        away_scores = manager.list(repeat(0, n_simuls))
        home_scores = manager.list(repeat(0, n_simuls))

        procs = []  #  1, 2, ..., 250 -> 25개 프로세서로 한 프로세서당 10개 simul
        for procID in range(n_procs):  # Simulation using multi-processing.
            begin = procID * (n_simuls // n_procs)
            end = (procID + 1) * (n_simuls // n_procs) if procID < n_procs - 1 else n_simuls
            proc = Process(target=proc_func, args=(away_scores, home_scores, begin, end))
            proc.start()
            procs.append(proc)

        for proc in procs:  # Wait for all processes to complete.
            proc.join()
            
        away_scores = list(away_scores)
        home_scores = list(home_scores)
        
    away_cnt, draw_cnt, home_cnt = 0, 0, 0
    for away_score in away_scores:
        for home_score in home_scores:
            if away_score > home_score:
                away_cnt += 1
            elif away_score < home_score:
                home_cnt += 1
            else:
                draw_cnt += 1
            
    # 예상 결과
    if away_cnt > home_cnt:
        pred = 'away'
    else:
        pred = 'home'
        
    # 실제 결과
    if this_game.iloc[-1]['AWAY_SCORE_CT'] > this_game.iloc[-1]['HOME_SCORE_CT']:
        real = 'away'
    elif this_game.iloc[-1]['AWAY_SCORE_CT'] < this_game.iloc[-1]['HOME_SCORE_CT']:
        real = 'home'
    else:
        real = 'draw'
        
    # 맞은 것 카운트
    if real != 'draw':
        total += 1
    if pred == real:
        trues += 1
    
    # 결과 출력
    f.write(f"실제-{this_game.iloc[-1]['AWAY_SCORE_CT']}:{this_game.iloc[-1]['HOME_SCORE_CT']} / ")
    f.write(f'예상-{np.mean(away_scores)}:{np.mean(home_scores)} / ')
    f.write(f'away:draw:home-{away_cnt/(n_simuls**2)}:{draw_cnt/(n_simuls**2)}:{home_cnt/(n_simuls**2)} / ')
    f.write(f'예측-{pred}')
    if (i+1) % 10 == 0:
        f.write(f' / {i+1}번 게임까지 정확도-{trues / total}')
    f.write('\n')
    f.flush()
    
    # Update the weights(model.coef_, model.intercept_)
    update_weights(this_game, model)
    
    # 타자, 투수 정보 추가
    for idx, row in this_game.iterrows():
        bat_id, pit_id = row['BAT_ID'], row['PIT_ID']
        event = row['EVENT_CD']
        
        if bat_id not in bat:
            bat[bat_id] = {key: 0 for key in pa_events + ['pa']}
        if pit_id not in pit:
            pit[pit_id] = {key: 0 for key in pa_events + ['pa']}
            
        bat[bat_id]['pa'] += 1
        bat[bat_id][event] += 1
        bat['average']['pa'] += 1
        bat['average'][event] += 1
        
        pit[pit_id]['pa'] += 1
        pit[pit_id][event] += 1
        pit['average']['pa'] += 1
        pit['average'][event] += 1
        
        if row['BAT_HOME_ID'] == 0:
            team_info[row['HOME_TEAM_ID']]['pa'] += 1
            team_info[row['HOME_TEAM_ID']][event] += 1
        else:
            team_info[row['AWAY_TEAM_ID']]['pa'] += 1
            team_info[row['AWAY_TEAM_ID']][event] += 1
        
f.close()  # 로그 파일 닫기

In [ ]:
trues / total